# Amazon so minimax bez alfa beta so funkcijata od iks tocka

In [1]:
from time import sleep
from plotly import graph_objects as go
import ipywidgets as ipw
from IPython import display
from copy import deepcopy

In [19]:
class Game:
    def __init__(self, state, player_1, player_2):
        self.N = len(state)
        self.state_original = state
        self.state = deepcopy(self.state_original)
        self.player_1 = {**player_1, 'symbol': 'S', 'role': 'MAX'}
        self.player_2 = {**player_2, 'symbol': 'P', 'role': 'MIN'}
        self.next_to_play = self.player_1
        self.symbols_fig = {'S': 'star', 'P': 'pentagon', 'x': 'x', '·': 'circle-open'}
        self.scores = {'S': 1, 'P': -1}
        self.create_ui()
        hbox = ipw.HBox([self.bt_reset, self.dashboard])
        display.display(ipw.VBox([hbox, self.fig]))
        self.reset()

    def reset(self, *args):
        self.h_calls = 0
        self.max_depth = 0
        self.next_to_play = self.player_1
        self.next_human_move = 'queen_moves'
        self.update_score('На ред е', self.next_to_play)
        if self.next_to_play['type'] == 'human':
            self.dashboard.value += ' -- помести ја кралицата'
        self.state = deepcopy(self.state_original)
        self.evaluated = {}
        self.fig.data[0].marker.symbol = self.convert_state_to_symbols()
        self.winner = 'keep_playing'
        self.initiate_turn()

    def create_ui(self):
        self.dashboard = ipw.HTML(description='Статус:', value='')
        self.bt_reset = ipw.Button(description='Ресетирај')
        self.bt_reset.on_click(self.reset)
        self.fig = self.create_fig()

    def create_fig(self):
        fig = go.FigureWidget()
        x = [x for y in range(self.N) for x in range(self.N)]
        y = [y for y in range(self.N) for x in range(self.N)]
        symbols = [self.symbols_fig[v] for row in self.state for v in row]
        fig.add_scatter(x=x, y=y, mode='markers', marker_size=48,
                        marker_symbol=symbols, marker_color='LightSkyBlue',
                        marker_line_width=6, marker_line_color='MediumPurple')
        fig.data[0].on_click(self.human_move)
        fig.update_xaxes(range=[-0.5, self.N - 0.5], dtick=1, title='x', side='top')
        fig.update_yaxes(
            range=[-0.5, self.N - 0.5], dtick=1, title='y', autorange='reversed')
        fig.update_layout(width=600, height=600, showlegend=False)
        return fig

    def convert_state_to_symbols(self):
        return [self.symbols_fig[v] for row in self.state for v in row]

    def initiate_turn(self):
        if 'human' not in [self.player_1['type'], self.player_2['type']]:
            while self.winner == 'keep_playing':
                self.ai_move()
        elif self.next_to_play['type'] == 'AI':
            self.ai_move()

    def ai_move(self):     
        self.dashboard.value += ' -- пресметува'
        state = tuple([tuple(row) for row in self.state])
        result, move = self.minimax(state, self.next_to_play['role'])
        qx, qy, px_move, py_move, px_shot, py_shot = move
        self.state[py_move][px_move] = self.state[qy][qx]
        self.state[qy][qx] = '·'
        self.state[py_shot][px_shot] = 'x'
        self.update_after_state_change()
        sleep(3)
        self.player_took_turn()

    def human_move(self, trace, points, selector):   
        x, y = points.xs[0], points.ys[0]
        keep_playing = self.winner == 'keep_playing'
        human_on_turn = self.next_to_play['type'] == 'human'
        if keep_playing and human_on_turn:
            if self.next_human_move == 'queen_moves':
                qx, qy = self.find_queen(self.state, self.next_to_play['symbol'])
                if (x, y) in list(self.possible_moves(self.state, qx, qy)):
                    self.state[y][x] = self.state[qy][qx]
                    self.state[qy][qx] = '·'
                    self.update_after_state_change()
                    self.next_human_move = 'queen_shoots'
                    self.dashboard.value += ' -- пукај'
            elif self.next_human_move == 'queen_shoots':
                qx, qy = self.find_queen(self.state, self.next_to_play['symbol'])
                if (x, y) in list(self.possible_moves(self.state, qx, qy)):
                    self.state[y][x] = 'x'
                    self.update_after_state_change()
                    self.next_human_move = 'queen_moves'
                    self.player_took_turn()
                    if self.next_to_play['type'] == 'AI':
                        self.ai_move()

    def update_after_state_change(self):   
        self.fig.data[0].marker.symbol = self.convert_state_to_symbols()

    def flip_next_player(self):
        if self.next_to_play == self.player_2:
            return self.player_1
        return self.player_2

    def player_took_turn(self):
        self.next_to_play = self.flip_next_player()
        self.winner = self.check_victory(self.state, self.next_to_play['symbol'])
        if self.winner != 'keep_playing':
            self.update_score('Победник е', self.flip_next_player())
            return
        self.update_score('На ред е', self.next_to_play)
        if self.next_to_play['type'] == 'human':
            self.dashboard.value += ' -- помести ја кралицата'

    def update_score(self, message, player):
        player_data = ' - '.join(list(player.values())[:-1])
        self.dashboard.value = f'{message} <b> {player_data} </b>.'

    def minimax(self, node, player, depth=0, node_count=0):
        winner = self.check_victory(node)
        if winner != 'keep_playing':
            return self.scores[winner], None, node_count + 1
    
        best_value = float('inf') if player == 'MIN' else float('-inf')
        best_move = None
    
        for child, move in self.expand_state(node, player):
            other_player = 'MIN' if player == 'MAX' else 'MAX'
            result, _, node_count = self.minimax(child, other_player, depth + 1, node_count + 1)
    
            if player == 'MIN':
                if result < best_value:
                    best_value = result
                    best_move = move
            else:  # player == 'MAX'
                if result > best_value:
                    best_value = result
                    best_move = move
    
        return best_value, best_move, node_count
    

    def find_queen(self, state, queen_symbol):
        for y in range(self.N):
            for x in range(self.N):
                if state[y][x] == queen_symbol:
                    return x, y

    def possible_moves(self, state, x, y):
        deltas = [
            (0, 1), (0, -1), (1, 0), (-1, 0),
            (1, 1), (1, -1), (-1, 1), (-1, -1)]
        for dx, dy in deltas:
            nx, ny = x + dx, y + dy
            while 0 <= nx < self.N and 0 <= ny < self.N:
                if state[ny][nx] == '·':
                    yield nx, ny
                else:
                    break
                nx += dx
                ny += dy

    def possible_shots(self, state, x, y):
        shots = [
            (0, 1), (0, -1), (1, 0), (-1, 0),
            (1, 1), (1, -1), (-1, 1), (-1, -1)]
        for dx, dy in shots:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.N and 0 <= ny < self.N:
                if state[ny][nx] == '·':
                    yield nx, ny

    def expand_state(self, state, player):
        symbol = 'S' if player == 'MAX' else 'P'
        qx, qy = self.find_queen(state, symbol)
        for px_move, py_move in self.possible_moves(state, qx, qy):
            state_after_move = list([list(row) for row in state])
            state_after_move[py_move][px_move] = symbol
            state_after_move[qy][qx] = '·'
            for px_shot, py_shot in self.possible_moves(state_after_move, px_move, py_move):
                state_after_shot = deepcopy(state_after_move)
                state_after_shot[py_shot][px_shot] = 'x'
                state_after_shot = tuple([tuple(row) for row in state_after_shot])
                yield state_after_shot, [qx, qy, px_move, py_move, px_shot, py_shot]

    def other_queen_symbol(self, queen_symbol):
        return 'P' if queen_symbol == 'S' else 'S'

    def check_victory(self, state, queen_to_move__symbol):
        queen_to_move__symbol
        qx, qy = self.find_queen(state, queen_to_move__symbol)
        if list(self.possible_moves(state, qx, qy)) == []:
            return self.other_queen_symbol(queen_to_move__symbol)
        return 'keep_playing'

In [20]:
initial_state = [
    ['·', '·', '·'],
    ['·', 'S', '·'],
    ['·', '·', 'P']
]
game = Game3(initial_state, {'name': 'Player 1', 'type': 'AI'}, {'name': 'Player 2', 'type': 'AI'})

Initiating turn for: Player 1
AI is calculating move...
Вкупно разгранети јазли: 1657107
AI chose move: [1, 1, 1, 2, 1, 1]
Winner: keep_playing
AI is calculating move...
Вкупно разгранети јазли: 4627
AI chose move: [2, 2, 2, 1, 2, 2]
Winner: keep_playing
AI is calculating move...
Вкупно разгранети јазли: 721
AI chose move: [1, 2, 0, 1, 0, 0]
Winner: keep_playing
AI is calculating move...
Вкупно разгранети јазли: 130
AI chose move: [2, 1, 2, 0, 2, 1]
Winner: keep_playing
AI is calculating move...
Вкупно разгранети јазли: 22
AI chose move: [0, 1, 0, 2, 0, 1]
Winner: keep_playing
AI is calculating move...
Вкупно разгранети јазли: 3
AI chose move: [2, 0, 1, 0, 2, 0]
Winner: keep_playing
AI is calculating move...
Вкупно разгранети јазли: 2
AI chose move: [0, 2, 1, 2, 0, 2]
Winner: S


# So alfa beta

In [21]:
class Game:
    def __init__(self, state, player_1, player_2):
        self.N = len(state)
        self.state_original = state
        self.state = deepcopy(self.state_original)
        self.player_1 = {**player_1, 'symbol': 'S', 'role': 'MAX'}
        self.player_2 = {**player_2, 'symbol': 'P', 'role': 'MIN'}
        self.next_to_play = self.player_1
        self.symbols_fig = {'S': 'star', 'P': 'pentagon', 'x': 'x', '·': 'circle-open'}
        self.scores = {'S': 1, 'P': -1}
        self.create_ui()
        hbox = ipw.HBox([self.bt_reset, self.dashboard])
        display.display(ipw.VBox([hbox, self.fig]))
        self.reset()

    def reset(self, *args):
        self.h_calls = 0
        self.max_depth = 0
        self.next_to_play = self.player_1
        self.next_human_move = 'queen_moves'
        self.update_score('На ред е', self.next_to_play)
        if self.next_to_play['type'] == 'human':
            self.dashboard.value += ' -- помести ја кралицата'
        self.state = deepcopy(self.state_original)
        self.evaluated = {}
        self.fig.data[0].marker.symbol = self.convert_state_to_symbols()
        self.winner = 'keep_playing'
        self.initiate_turn()

    def create_ui(self):
        self.dashboard = ipw.HTML(description='Статус:', value='')
        self.bt_reset = ipw.Button(description='Ресетирај')
        self.bt_reset.on_click(self.reset)
        self.fig = self.create_fig()

    def create_fig(self):
        fig = go.FigureWidget()
        x = [x for y in range(self.N) for x in range(self.N)]
        y = [y for y in range(self.N) for x in range(self.N)]
        symbols = [self.symbols_fig[v] for row in self.state for v in row]
        fig.add_scatter(x=x, y=y, mode='markers', marker_size=48,
                        marker_symbol=symbols, marker_color='LightSkyBlue',
                        marker_line_width=6, marker_line_color='MediumPurple')
        fig.data[0].on_click(self.human_move)
        fig.update_xaxes(range=[-0.5, self.N - 0.5], dtick=1, title='x', side='top')
        fig.update_yaxes(
            range=[-0.5, self.N - 0.5], dtick=1, title='y', autorange='reversed')
        fig.update_layout(width=600, height=600, showlegend=False)
        return fig

    def convert_state_to_symbols(self):
        return [self.symbols_fig[v] for row in self.state for v in row]

    def initiate_turn(self):
        if 'human' not in [self.player_1['type'], self.player_2['type']]:
            while self.winner == 'keep_playing':
                self.ai_move()
        elif self.next_to_play['type'] == 'AI':
            self.ai_move()

    def ai_move(self):     
        self.dashboard.value += ' -- пресметува'
        state = tuple([tuple(row) for row in self.state])
        result, move = self.minimax(state, self.next_to_play['role'])
        qx, qy, px_move, py_move, px_shot, py_shot = move
        self.state[py_move][px_move] = self.state[qy][qx]
        self.state[qy][qx] = '·'
        self.state[py_shot][px_shot] = 'x'
        self.update_after_state_change()
        sleep(3)
        self.player_took_turn()

    def human_move(self, trace, points, selector):   
        x, y = points.xs[0], points.ys[0]
        keep_playing = self.winner == 'keep_playing'
        human_on_turn = self.next_to_play['type'] == 'human'
        if keep_playing and human_on_turn:
            if self.next_human_move == 'queen_moves':
                qx, qy = self.find_queen(self.state, self.next_to_play['symbol'])
                if (x, y) in list(self.possible_moves(self.state, qx, qy)):
                    self.state[y][x] = self.state[qy][qx]
                    self.state[qy][qx] = '·'
                    self.update_after_state_change()
                    self.next_human_move = 'queen_shoots'
                    self.dashboard.value += ' -- пукај'
            elif self.next_human_move == 'queen_shoots':
                qx, qy = self.find_queen(self.state, self.next_to_play['symbol'])
                if (x, y) in list(self.possible_moves(self.state, qx, qy)):
                    self.state[y][x] = 'x'
                    self.update_after_state_change()
                    self.next_human_move = 'queen_moves'
                    self.player_took_turn()
                    if self.next_to_play['type'] == 'AI':
                        self.ai_move()

    def update_after_state_change(self):   
        self.fig.data[0].marker.symbol = self.convert_state_to_symbols()

    def flip_next_player(self):
        if self.next_to_play == self.player_2:
            return self.player_1
        return self.player_2

    def player_took_turn(self):
        self.next_to_play = self.flip_next_player()
        self.winner = self.check_victory(self.state, self.next_to_play['symbol'])
        if self.winner != 'keep_playing':
            self.update_score('Победник е', self.flip_next_player())
            return
        self.update_score('На ред е', self.next_to_play)
        if self.next_to_play['type'] == 'human':
            self.dashboard.value += ' -- помести ја кралицата'

    def update_score(self, message, player):
        player_data = ' - '.join(list(player.values())[:-1])
        self.dashboard.value = f'{message} <b> {player_data} </b>.'

    def minimax(self, node, player, alpha=-2, beta=2, depth=0, node_count=0):
        winner = self.check_victory(node, 'S' if player == 'MAX' else 'P')
        if winner != 'keep_playing':
            return self.scores[winner], None, node_count + 1  # Три вредности
    
        best_value = 2 if player == 'MIN' else -2
        best_move = None
    
        for child, move in self.expand_state(node, player):
            other_player = 'MIN' if player == 'MAX' else 'MAX'
            # Три вредности од рекурзивен повик
            result, _, node_count = self.minimax(child, other_player, alpha, beta, depth + 1, node_count + 1)
            if player == 'MIN':
                if result <= alpha:
                    return result, best_move, node_count
                if result < beta:
                    beta = result
                if result < best_value:
                    best_value = result
                    best_move = move
            elif player == 'MAX':
                if result >= beta:
                    return result, best_move, node_count
                if result > alpha:
                    alpha = result
                if result > best_value:
                    best_value = result
                    best_move = move
        print(f"Izbrojav:{node_count}")
        return best_value, best_move, node_count
    

    def find_queen(self, state, queen_symbol):
        for y in range(self.N):
            for x in range(self.N):
                if state[y][x] == queen_symbol:
                    return x, y

    def possible_moves(self, state, x, y):
        deltas = [
            (0, 1), (0, -1), (1, 0), (-1, 0),
            (1, 1), (1, -1), (-1, 1), (-1, -1)]
        for dx, dy in deltas:
            nx, ny = x + dx, y + dy
            while 0 <= nx < self.N and 0 <= ny < self.N:
                if state[ny][nx] == '·':
                    yield nx, ny
                else:
                    break
                nx += dx
                ny += dy

    def possible_shots(self, state, x, y):
        shots = [
            (0, 1), (0, -1), (1, 0), (-1, 0),
            (1, 1), (1, -1), (-1, 1), (-1, -1)]
        for dx, dy in shots:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.N and 0 <= ny < self.N:
                if state[ny][nx] == '·':
                    yield nx, ny

    def expand_state(self, state, player):
        symbol = 'S' if player == 'MAX' else 'P'
        qx, qy = self.find_queen(state, symbol)
        for px_move, py_move in self.possible_moves(state, qx, qy):
            state_after_move = list([list(row) for row in state])
            state_after_move[py_move][px_move] = symbol
            state_after_move[qy][qx] = '·'
            for px_shot, py_shot in self.possible_moves(state_after_move, px_move, py_move):
                state_after_shot = deepcopy(state_after_move)
                state_after_shot[py_shot][px_shot] = 'x'
                state_after_shot = tuple([tuple(row) for row in state_after_shot])
                yield state_after_shot, [qx, qy, px_move, py_move, px_shot, py_shot]

    def other_queen_symbol(self, queen_symbol):
        return 'P' if queen_symbol == 'S' else 'S'

    def check_victory(self, state, queen_to_move__symbol):
        queen_to_move__symbol
        qx, qy = self.find_queen(state, queen_to_move__symbol)
        if list(self.possible_moves(state, qx, qy)) == []:
            return self.other_queen_symbol(queen_to_move__symbol)
        return 'keep_playing'

In [22]:
initial_state = [
    ['·', '·', '·'],
    ['·', 'S', '·'],
    ['·', '·', 'P']
]
game = Game3(initial_state, {'name': 'Player 1', 'type': 'AI'}, {'name': 'Player 2', 'type': 'AI'})

Initiating turn for: Player 1
AI is calculating move...
Вкупно разгранети јазли: 1657107
AI chose move: [1, 1, 1, 2, 1, 1]
Winner: keep_playing
AI is calculating move...
Вкупно разгранети јазли: 4627
AI chose move: [2, 2, 2, 1, 2, 2]
Winner: keep_playing
AI is calculating move...
Вкупно разгранети јазли: 721
AI chose move: [1, 2, 0, 1, 0, 0]
Winner: keep_playing
AI is calculating move...
Вкупно разгранети јазли: 130
AI chose move: [2, 1, 2, 0, 2, 1]
Winner: keep_playing
AI is calculating move...
Вкупно разгранети јазли: 22
AI chose move: [0, 1, 0, 2, 0, 1]
Winner: keep_playing
AI is calculating move...
Вкупно разгранети јазли: 3
AI chose move: [2, 0, 1, 0, 2, 0]
Winner: keep_playing
AI is calculating move...
Вкупно разгранети јазли: 2
AI chose move: [0, 2, 1, 2, 0, 2]
Winner: S
